# DATAFRAME

In [1]:
#r "nuget:Microsoft.Data.Analysis,0.4.0"
#r "nuget:Microsoft.ML,1.5.2"
using Microsoft.AspNetCore.Html;
using Microsoft.Data.Analysis;
using Microsoft.DotNet.Interactive.Formatting;

### DataFrame object formatter (included in Microsoft.Data.Analysis, probably will be released in with the next version, 0.5.0)

In [1]:
public static void RegisterDataFrame()
{
    Formatter.Register<DataFrame>((df, writer) =>
    {
        const int MAX = 10000;
        const int SIZE = 10;

        var uniqueId = DateTime.Now.Ticks;

        var header = new List<IHtmlContent>
        {
            th(i("index"))
        };
        header.AddRange(df.Columns.Select(c => (IHtmlContent)th(c.Name)));

        if (df.Rows.Count > SIZE)
        {
            var maxMessage = df.Rows.Count > MAX ? $" (showing a max of {MAX} rows)" : string.Empty;
            var title = h3[style: "text-align: center;"]($"DataFrame - {df.Rows.Count} rows {maxMessage}");

            // table body
            var maxRows = Math.Min(MAX, df.Rows.Count);
            var rows = new List<List<IHtmlContent>>();
            for (var index = 0; index < maxRows; index++)
            {
                var cells = new List<IHtmlContent>
                {
                    td(i((index)))
                };
                foreach (var obj in df.Rows[index])
                {
                    cells.Add(td(obj));
                }
                rows.Add(cells);
            }

            //navigator      
            var footer = new List<IHtmlContent>();
            BuildHideRowsScript(uniqueId);

            var paginateScriptFirst = BuildHideRowsScript(uniqueId) + GotoPageIndex(uniqueId, 0) + BuildPageScript(uniqueId, SIZE);
            footer.Add(button[style: "margin: 2px;", onclick: paginateScriptFirst]("⏮"));

            var paginateScriptPrevTen = BuildHideRowsScript(uniqueId) + UpdatePageIndex(uniqueId, -10, (maxRows - 1) / SIZE) + BuildPageScript(uniqueId, SIZE);
            footer.Add(button[style: "margin: 2px;", onclick: paginateScriptPrevTen]("⏪"));

            var paginateScriptPrev = BuildHideRowsScript(uniqueId) + UpdatePageIndex(uniqueId, -1, (maxRows - 1) / SIZE) + BuildPageScript(uniqueId, SIZE);
            footer.Add(button[style: "margin: 2px;", onclick: paginateScriptPrev]("◀️"));

            footer.Add(b[style: "margin: 2px;"]("Page"));
            footer.Add(b[id: $"page_{uniqueId}", style: "margin: 2px;"]("1"));

            var paginateScriptNext = BuildHideRowsScript(uniqueId) + UpdatePageIndex(uniqueId, 1, (maxRows - 1) / SIZE) + BuildPageScript(uniqueId, SIZE);
            footer.Add(button[style: "margin: 2px;", onclick: paginateScriptNext]("▶️"));

            var paginateScriptNextTen = BuildHideRowsScript(uniqueId) + UpdatePageIndex(uniqueId, 10, (maxRows - 1) / SIZE) + BuildPageScript(uniqueId, SIZE);
            footer.Add(button[style: "margin: 2px;", onclick: paginateScriptNextTen]("⏩"));

            var paginateScriptLast = BuildHideRowsScript(uniqueId) + GotoPageIndex(uniqueId, (maxRows - 1) / SIZE) + BuildPageScript(uniqueId, SIZE);
            footer.Add(button[style: "margin: 2px;", onclick: paginateScriptLast]("⏭️"));

            //table
            var t = table[id: $"table_{uniqueId}"](
                caption(title),
                thead(tr(header)),
                tbody(rows.Select(r => tr[style: "display: none"](r))),
                tfoot(tr(td[colspan: df.Columns.Count + 1, style: "text-align: center;"](footer)))
            );
            writer.Write(t);

            //show first page
            writer.Write($"<script>{BuildPageScript(uniqueId, SIZE)}</script>");
        }
        else
        {
            var rows = new List<List<IHtmlContent>>();
            for (var index = 0; index < df.Rows.Count; index++)
            {
                var cells = new List<IHtmlContent>
                {
                    td(i((index)))
                };
                foreach (var obj in df.Rows[index])
                {
                    cells.Add(td(obj));
                }
                rows.Add(cells);
            }

            //table
            var t = table[id: $"table_{uniqueId}"](
                thead(tr(header)),
                tbody(rows.Select(r => tr(r)))
            );
            writer.Write(t);
        }
    }, "text/html");
}

private static string BuildHideRowsScript(long uniqueId)
{
    var script = $"var allRows = document.querySelectorAll('#table_{uniqueId} tbody tr:nth-child(n)'); ";
    script += "for (let i = 0; i < allRows.length; i++) { allRows[i].style.display='none'; } ";
    return script;
}

private static string BuildPageScript(long uniqueId, int size)
{
    var script = $"var page = parseInt(document.querySelector('#page_{uniqueId}').innerHTML) - 1; ";
    script += $"var pageRows = document.querySelectorAll(`#table_{uniqueId} tbody tr:nth-child(n + ${{page * {size} + 1 }})`); ";
    script += $"for (let j = 0; j < {size}; j++) {{ pageRows[j].style.display='table-row'; }} ";
    return script;
}

private static string GotoPageIndex(long uniqueId, long page)
{
    var script = $"document.querySelector('#page_{uniqueId}').innerHTML = {page + 1}; ";
    return script;
}

private static string UpdatePageIndex(long uniqueId, int step, long maxPage)
{
    var script = $"var page = parseInt(document.querySelector('#page_{uniqueId}').innerHTML) - 1; ";
    script += $"page = parseInt(page) + parseInt({step}); ";
    script += $"page = page < 0 ? 0 : page; ";
    script += $"page = page > {maxPage} ? {maxPage} : page; ";
    script += $"document.querySelector('#page_{uniqueId}').innerHTML = page + 1; ";
    return script;
}

RegisterDataFrame();

### Load data into data frame

In [1]:
const string SENSORS_DATASET_PATH = "./sensors.csv";
var sensorsData = DataFrame.LoadCsv(SENSORS_DATASET_PATH);

display(sensorsData);

index,Temperature,Luminosity,Infrared,Distance,CreatedAt
0,24.77,63.96,0,242.96,2020-10-08T19:38:18.7814575Z
1,23.67,64.06,0,301.72,2020-10-08T19:38:18.7814575Z
2,24.12,69.14,0,350.8,2020-10-08T19:38:19.7814575Z
3,23.8,65.62,0,13.86,2020-10-08T19:38:19.7814575Z
4,23.8,66.41,0,290.37,2020-10-08T19:38:19.7814575Z
5,24.77,68.85,0,382.72,2020-10-08T19:38:20.7814575Z
6,24.63,4.1,0,153.67,2020-10-08T19:38:30.7814575Z
7,24.6,6.05,0,154.49,2020-10-08T19:38:30.7814575Z
8,24.63,4.1,0,400,2020-10-08T19:38:31.7814575Z
9,24.31,3.91,0,5.13,2020-10-08T19:38:31.7814575Z


In [1]:
const string LABELS_DATASET_PATH = "./labels.csv";
var labelsData = DataFrame.LoadCsv(LABELS_DATASET_PATH);

display(labelsData);

index,CreatedAt,Score,Label
0,2020-10-08T19:38:18.7814575Z,98.33,FlashLight
1,2020-10-08T19:38:18.7814575Z,78.23,FlashLight
2,2020-10-08T19:38:19.7814575Z,93.21,FlashLight
3,2020-10-08T19:38:19.7814575Z,67.87,FlashLight
4,2020-10-08T19:38:30.7814575Z,76.77,FlashLight
5,2020-10-08T19:38:31.7814575Z,89.54,Day
6,2020-10-08T19:38:31.7814575Z,99.12,Day
7,2020-10-08T19:38:34.7814575Z,95.54,Day
8,2020-10-08T19:38:34.7814575Z,96.65,Lighter
9,2020-10-08T19:38:35.7814575Z,74.56,Lighter


### Prepare sensors dataframe


In [1]:
var sensorsCreatedAt = sensorsData.Columns["CreatedAt"] as IEnumerable<string>;
var sensorsTicksList = sensorsCreatedAt.ToList().Select(d => DateTime.Parse(d.ToString()).Ticks / 10000000);
var sensorsTicks = new PrimitiveDataFrameColumn<long>("Id", sensorsTicksList);
var sensorsDataPlus = new DataFrame(sensorsData.Columns["Temperature"], sensorsData.Columns["Luminosity"], sensorsData.Columns["Infrared"], sensorsData.Columns["Distance"], sensorsData.Columns["CreatedAt"], sensorsTicks);

display(sensorsDataPlus);

index,Temperature,Luminosity,Infrared,Distance,CreatedAt,Id
0,24.77,63.96,0,242.96,2020-10-08T19:38:18.7814575Z,63737789898
1,23.67,64.06,0,301.72,2020-10-08T19:38:18.7814575Z,63737789898
2,24.12,69.14,0,350.8,2020-10-08T19:38:19.7814575Z,63737789899
3,23.8,65.62,0,13.86,2020-10-08T19:38:19.7814575Z,63737789899
4,23.8,66.41,0,290.37,2020-10-08T19:38:19.7814575Z,63737789899
5,24.77,68.85,0,382.72,2020-10-08T19:38:20.7814575Z,63737789900
6,24.63,4.1,0,153.67,2020-10-08T19:38:30.7814575Z,63737789910
7,24.6,6.05,0,154.49,2020-10-08T19:38:30.7814575Z,63737789910
8,24.63,4.1,0,400,2020-10-08T19:38:31.7814575Z,63737789911
9,24.31,3.91,0,5.13,2020-10-08T19:38:31.7814575Z,63737789911


### Prepare labels dataframe

In [1]:
var labelsCreatedAt = labelsData.Columns["CreatedAt"] as IEnumerable<string>;
var labelsTicksList = labelsCreatedAt.ToList().Select(d => DateTime.Parse(d.ToString()).Ticks / 10000000);
var labelsTicks = new PrimitiveDataFrameColumn<long>("Id", labelsTicksList);
var labelsDataPlus = new DataFrame(labelsTicks, labelsData.Columns["CreatedAt"], labelsData.Columns["Label"], labelsData.Columns["Score"]);

display(labelsDataPlus);

index,Id,CreatedAt,Label,Score
0,63737789898,2020-10-08T19:38:18.7814575Z,FlashLight,98.33
1,63737789898,2020-10-08T19:38:18.7814575Z,FlashLight,78.23
2,63737789899,2020-10-08T19:38:19.7814575Z,FlashLight,93.21
3,63737789899,2020-10-08T19:38:19.7814575Z,FlashLight,67.87
4,63737789910,2020-10-08T19:38:30.7814575Z,FlashLight,76.77
5,63737789911,2020-10-08T19:38:31.7814575Z,Day,89.54
6,63737789911,2020-10-08T19:38:31.7814575Z,Day,99.12
7,63737789914,2020-10-08T19:38:34.7814575Z,Day,95.54
8,63737789914,2020-10-08T19:38:34.7814575Z,Lighter,96.65
9,63737789915,2020-10-08T19:38:35.7814575Z,Lighter,74.56


### Merge sensors dataframe and labels dataframe into 'merged' dataframe

In [1]:
var merged = sensorsDataPlus.Merge<long>(labelsDataPlus, "Id", "Id", "_sensor", "_label", JoinAlgorithm.FullOuter);

display(merged);

index,Temperature,Luminosity,Infrared,Distance,CreatedAt_sensor,Id_sensor,Id_label,CreatedAt_label,Label,Score
0,24.77,63.96,0,242.96,2020-10-08T19:38:18.7814575Z,63737789898,63737789898,2020-10-08T19:38:18.7814575Z,FlashLight,98.33
1,24.77,63.96,0,242.96,2020-10-08T19:38:18.7814575Z,63737789898,63737789898,2020-10-08T19:38:18.7814575Z,FlashLight,78.23
2,23.67,64.06,0,301.72,2020-10-08T19:38:18.7814575Z,63737789898,63737789898,2020-10-08T19:38:18.7814575Z,FlashLight,98.33
3,23.67,64.06,0,301.72,2020-10-08T19:38:18.7814575Z,63737789898,63737789898,2020-10-08T19:38:18.7814575Z,FlashLight,78.23
4,24.12,69.14,0,350.8,2020-10-08T19:38:19.7814575Z,63737789899,63737789899,2020-10-08T19:38:19.7814575Z,FlashLight,93.21
5,24.12,69.14,0,350.8,2020-10-08T19:38:19.7814575Z,63737789899,63737789899,2020-10-08T19:38:19.7814575Z,FlashLight,67.87
6,23.8,65.62,0,13.86,2020-10-08T19:38:19.7814575Z,63737789899,63737789899,2020-10-08T19:38:19.7814575Z,FlashLight,93.21
7,23.8,65.62,0,13.86,2020-10-08T19:38:19.7814575Z,63737789899,63737789899,2020-10-08T19:38:19.7814575Z,FlashLight,67.87
8,23.8,66.41,0,290.37,2020-10-08T19:38:19.7814575Z,63737789899,63737789899,2020-10-08T19:38:19.7814575Z,FlashLight,93.21
9,23.8,66.41,0,290.37,2020-10-08T19:38:19.7814575Z,63737789899,63737789899,2020-10-08T19:38:19.7814575Z,FlashLight,67.87


### Clean 'merged' dataframe to 'cleaned' dataframe and save it to csv file

In [1]:
var cleaned = merged.DropNulls();

display(cleaned);



index,Temperature,Luminosity,Infrared,Distance,CreatedAt_sensor,Id_sensor,Id_label,CreatedAt_label,Label,Score
0,24.77,63.96,0,242.96,2020-10-08T19:38:18.7814575Z,63737789898,63737789898,2020-10-08T19:38:18.7814575Z,FlashLight,98.33
1,24.77,63.96,0,242.96,2020-10-08T19:38:18.7814575Z,63737789898,63737789898,2020-10-08T19:38:18.7814575Z,FlashLight,78.23
2,23.67,64.06,0,301.72,2020-10-08T19:38:18.7814575Z,63737789898,63737789898,2020-10-08T19:38:18.7814575Z,FlashLight,98.33
3,23.67,64.06,0,301.72,2020-10-08T19:38:18.7814575Z,63737789898,63737789898,2020-10-08T19:38:18.7814575Z,FlashLight,78.23
4,24.12,69.14,0,350.8,2020-10-08T19:38:19.7814575Z,63737789899,63737789899,2020-10-08T19:38:19.7814575Z,FlashLight,93.21
5,24.12,69.14,0,350.8,2020-10-08T19:38:19.7814575Z,63737789899,63737789899,2020-10-08T19:38:19.7814575Z,FlashLight,67.87
6,23.8,65.62,0,13.86,2020-10-08T19:38:19.7814575Z,63737789899,63737789899,2020-10-08T19:38:19.7814575Z,FlashLight,93.21
7,23.8,65.62,0,13.86,2020-10-08T19:38:19.7814575Z,63737789899,63737789899,2020-10-08T19:38:19.7814575Z,FlashLight,67.87
8,23.8,66.41,0,290.37,2020-10-08T19:38:19.7814575Z,63737789899,63737789899,2020-10-08T19:38:19.7814575Z,FlashLight,93.21
9,23.8,66.41,0,290.37,2020-10-08T19:38:19.7814575Z,63737789899,63737789899,2020-10-08T19:38:19.7814575Z,FlashLight,67.87


### WriteCsv method (included in Microsoft.Data.Analysis, probably will be released in with the next version, 0.5.0)

In [1]:
using System.Globalization;
using System.IO;

private const int DefaultStreamReaderBufferSize = 1024;

public static void WriteCsv(DataFrame dataFrame, Stream csvStream,
                           char separator = ',', bool header = true,
                           Encoding encoding = null, CultureInfo cultureInfo = null)
{
    if (cultureInfo is null)
    {
        cultureInfo = CultureInfo.CurrentCulture;
    }

    if (cultureInfo.NumberFormat.NumberDecimalSeparator.Equals(separator.ToString()))
    {
        throw new ArgumentException("Decimal separator cannot match the column separator");
    }

    if (encoding is null)
    {
        encoding = Encoding.ASCII;
    }

    using (StreamWriter csvFile = new StreamWriter(csvStream, encoding, bufferSize: DefaultStreamReaderBufferSize, leaveOpen: true))
    {
        if (dataFrame != null)
        {
            var columnNames = new List<string>();
            
            foreach (var col in dataFrame.Columns)
            {
                columnNames.Add(col.Name);
            }

            if (header)
            {
                var headerColumns = string.Join(separator.ToString(), columnNames);
                csvFile.WriteLine(headerColumns);
            }

            var record = new StringBuilder();

            foreach (var row in dataFrame.Rows)
            {
                bool firstRow = true;
                foreach (var cell in row)
                {
                    if (!firstRow)
                    {
                        record.Append(separator);
                    }
                    else
                    {
                        firstRow = false;
                    }

                    Type t = cell?.GetType();

                    if (t == typeof(bool))
                    {
                        record.AppendFormat(cultureInfo, "{0}", cell);
                        continue;
                    }

                    if (t == typeof(float))
                    {
                        record.AppendFormat(cultureInfo, "{0:G9}", cell);
                        continue;
                    }

                    if (t == typeof(double))
                    {
                        record.AppendFormat(cultureInfo, "{0:G17}", cell);
                        continue;
                    }

                    if (t == typeof(decimal))
                    {
                        record.AppendFormat(cultureInfo, "{0:G31}", cell);
                        continue;
                    }

                    record.Append(cell);
                }

                csvFile.WriteLine(record);

                record.Clear();
            }
        }
    }
}

public static void WriteCsv(DataFrame dataFrame, string path,
                            char separator = ',', bool header = true,
                            Encoding encoding = null, CultureInfo cultureInfo = null)
{
    using (System.IO.FileStream csvStream = new FileStream(path, FileMode.Create))
    {
        WriteCsv(dataFrame: dataFrame, csvStream: csvStream,
                    separator: separator, header: header,
                    encoding: encoding, cultureInfo: cultureInfo);
    }
}

In [1]:
WriteCsv(cleaned, "merged.csv");